In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [21]:
data = pd.read_csv('../../data/Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [22]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)


In [23]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [24]:
One_encoder = OneHotEncoder(drop='first',sparse_output=False)
geo_encoder= One_encoder.fit_transform(data[['Geography']])

In [25]:
geo_df = pd.DataFrame(geo_encoder, columns=One_encoder.get_feature_names_out(['Geography']))

In [26]:
data = pd.concat([data.drop(['Geography'], axis=1), geo_df], axis=1)

In [27]:
X= data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

In [29]:
X_train ,X_test ,y_train ,y_test= train_test_split(X,y,test_size=0.2,random_state=42)
scaler= StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [30]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(One_encoder,file)    

with open('scaler.pkl','wb')as file:
    pickle.dump(scaler,file)

In [33]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [42]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

c:\Users\OMAR\Desktop\projects\Ai-ML-DL-NLP\Ann_Project\venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [43]:
model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mae'])

In [44]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

In [45]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

log_dir="regression/fit/" + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback= TensorBoard(log_dir=log_dir,histogram_freq=1)

In [46]:
early_stopping_callback= EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [47]:
history = model.fit(
    X_train,y_train, validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 13337486336.0000 - mae: 100040.8828 - val_loss: 13015811072.0000 - val_mae: 98562.3047
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13513306112.0000 - mae: 100914.2812 - val_loss: 12753622016.0000 - val_mae: 97234.2266
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 12880211968.0000 - mae: 97864.6953 - val_loss: 12054332416.0000 - val_mae: 93694.4531
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 12046968832.0000 - mae: 93695.2656 - val_loss: 10856600576.0000 - val_mae: 87646.2344
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10633537536.0000 - mae: 86713.9609 - val_loss: 9267900416.0000 - val_mae: 79679.9766
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9168069632.0000 - mae: 79581.7422 - val_loss: 7530757632.0000 - val_mae: 70991.5156
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 7416471552.0000 - mae: 70875.8828 - val_loss: 5947730944.0

In [50]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [53]:
%tensorboard --logdir regression/fit

Reusing TensorBoard on port 6007 (pid 20760), started 0:00:06 ago. (Use '!kill 20760' to kill it.)

In [54]:
model.save('regression_model.h5')